In [ ]:
# General Libs
import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import RandomRotation, RandomWidth, RandomHeight, RandomZoom, RandomBrightness, RandomContrast, GaussianNoise, RandomFlip, Rescaling, Resizing, RandomCrop
from keras_cv.layers import RandomShear, RandomSaturation
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import os
from PIL import Image
%matplotlib inline

### Aplica Data Augmentation Layer (Keras 3.x e KerasCV) $\\$ Em imagens de uma pasta e salva em outra pasta

In [ ]:
# Data Augmentation Layer
im_size = (256, 256)  # Define the desired image size
data_augmentation = tf.keras.Sequential([
    RandomSaturation((0.4, 0.6)),  # Adjust saturation
    RandomRotation(0.15, fill_mode='nearest', fill_value=0),  # Random rotation
    RandomWidth(0.01),  # Width shift
    RandomHeight(0.01),  # Height shift
    RandomShear(0.01, fill_mode='nearest', fill_value=0),  # Shear
    RandomZoom(0.01, 0.01, fill_mode='nearest', fill_value=0),  # Zoom
    RandomBrightness(factor=0.1),  # Brightness
    RandomContrast(factor=0.1),  # Contrast
    GaussianNoise(0.1),  # Gaussian noise
    RandomFlip(mode="horizontal_and_vertical"),  # Random flip
    tf.keras.layers.Resizing(im_size[0], im_size[1]),  # Resize to im_size
])

# Directories
input_dir = 'C:\\cascade\\frutas\\p_train_old'
output_dir = 'C:\\cascade\\frutas\\banana_train\\p_train_aug'
total_augmented_images = 40000  # Total number of augmented images to generate

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to process and save images
def process_and_save_image(image_path, output_dir, num_augmented, img_counter):
    # Open the image using PIL
    image = Image.open(image_path)
    img_array = np.array(image)  # Convert to numpy array

    # Create a TensorFlow tensor from the numpy array
    img_tensor = tf.convert_to_tensor(img_array)

    for i in range(num_augmented):
        # Apply data augmentation
        augmented_img = data_augmentation(tf.expand_dims(img_tensor, axis=0))  # Add batch dimension
        
        # Convert the augmented tensor back to a PIL Image
        augmented_img = Image.fromarray(augmented_img[0].numpy().astype(np.uint8))  # Remove batch dimension
        
        # Save the augmented image to the output directory
        augmented_img.save(os.path.join(output_dir, f"augmented_image_{img_counter}.jpg"))
        img_counter += 1

    return img_counter

# Function to iterate over the image directory and process each image
def process_image_directory(input_dir, output_dir, total_augmented_images):
    img_filenames = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    num_original_images = len(img_filenames)

    # Calculate how many images to generate per original image
    images_per_file = total_augmented_images // num_original_images
    extra_images = total_augmented_images % num_original_images  # Handle remainder
    
    img_counter = 0
    while img_counter < total_augmented_images:
        for img_filename in img_filenames:
            img_path = os.path.join(input_dir, img_filename)

            # Adjust for any extra images that need to be generated
            num_augmented = images_per_file + 1 if extra_images > 0 else images_per_file
            extra_images = max(0, extra_images - 1)  # Decrease extra count after using it

            img_counter = process_and_save_image(img_path, output_dir, num_augmented, img_counter)
            
            # Stop if we've reached the target number of augmented images
            if img_counter >= total_augmented_images:
                break

# Process the input directory and save results
process_image_directory(input_dir, output_dir, total_augmented_images)

print(f"Processing complete. Augmented images saved in: {output_dir}")


### Gera Patches (Crop) de Imagens grandes com Data Augmentation Layer (Keras 3.x e KerasCV) $\\$ Em imagens de uma pasta e salva em outra pasta

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

# Directories
input_dir = 'C:\\cascade\\frutas\\banana_test\\n'
output_dir = 'C:\\cascade\\frutas\\banana_test\\n_crop'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to process and save images
def process_and_save_image(image_path, output_dir, crop_size=(256, 256)):
    # Open the image using PIL
    image = Image.open(image_path)
    img_array = np.array(image)  # Convert to numpy array
    img_height, img_width = img_array.shape[:2]  # Get image dimensions

    # If the image is <= 256x256, just copy it without changes
    if img_height <= crop_size[0] and img_width <= crop_size[1]:
        image.save(os.path.join(output_dir, os.path.basename(image_path)))
    else:
        # Create a TensorFlow tensor from the numpy array
        img_tensor = tf.convert_to_tensor(img_array)

        # Calculate how many crops to apply (1 crop if larger, 2 crops if 2x larger)
        crop_times = 1 if (img_height <= crop_size[0] * 2 and img_width <= crop_size[1] * 2) else 2

        for i in range(crop_times):
            # Ensure the image is larger than the crop size
            if img_height > crop_size[0] and img_width > crop_size[1]:
                # Apply random cropping
                cropped_img = tf.image.random_crop(img_tensor, size=[crop_size[0], crop_size[1], img_tensor.shape[2]])

                # Convert the cropped tensor back to a PIL Image
                cropped_img = Image.fromarray(cropped_img.numpy())

                # Save the cropped image to the output directory
                cropped_img.save(os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_crop_{i}.jpg"))
            else:
                print(f"Image {image_path} is not large enough for cropping.")

# Function to iterate over the image directory and process each image
def process_image_directory(input_dir, output_dir):
    for img_filename in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_filename)
        
        # Only process image files (skip non-image files)
        if img_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            process_and_save_image(img_path, output_dir)

# Process the input directory and save results
process_image_directory(input_dir, output_dir)

print(f"Processing complete. Cropped images saved in: {output_dir}")
